In [1]:
import pandas as pd

pd.set_option('display.max_columns', None)

In [2]:
rating = pd.read_csv('./csv/glicko_rating.csv', parse_dates=['last_game_date'])
rating.shape

(606, 9)

In [4]:
rating

,Team,League,Win,Loss,WinRate,Streak,r,RD,last_game_date
0,Gen.G,LCK,568,386,0.595388,4,2187.255923,121.438602,2023-02-09 12:37:00
1,T1,LCK,859,439,0.661787,3,2122.548742,138.836229,2023-02-08 12:32:00
2,Samsung White,LTC,94,42,0.691176,1,2009.034081,129.125220,2014-10-19 09:44:00
3,JD Gaming,LPL,385,284,0.575486,7,1990.271298,222.580624,2023-02-09 12:06:00
4,Dplus KIA,LCK,416,174,0.705085,-2,1941.128447,122.457776,2023-02-09 10:03:00
...,...,...,...,...,...,...,...,...,...
601,Bencheados,CLS,7,31,0.184211,-6,325.064140,300.660297,2017-03-30 23:23:00
602,V3 Esports,LJL,117,145,0.446565,-6,321.256428,250.201348,2023-02-05 12:36:00
603,Flash Wolves,RR,403,292,0.579856,-78,287.526103,308.107825,2019-09-07 09:01:00
604,Gravitas,LCO,37,124,0.229814,-22,212.771879,170.229248,2022-08-09 09:16:00


In [5]:
rating.loc[(rating['League'] == 'LCK') & (rating['last_game_date'].dt.year == 2023)]

,Team,League,Win,Loss,WinRate,Streak,r,RD,last_game_date
0,Gen.G,LCK,568,386,0.595388,4,2187.255923,121.438602,2023-02-09 12:37:00
1,T1,LCK,859,439,0.661787,3,2122.548742,138.836229,2023-02-08 12:32:00
4,Dplus KIA,LCK,416,174,0.705085,-2,1941.128447,122.457776,2023-02-09 10:03:00
5,Liiv SANDBOX,LCK,201,215,0.483173,1,1926.325018,92.311851,2023-02-10 13:14:00
6,KT Rolster,LCK,592,411,0.590229,2,1878.319323,103.767547,2023-02-09 10:03:00
8,Hanwha Life Esports,LCK,404,411,0.495706,1,1867.243762,109.264336,2023-02-10 10:00:00
9,Kwangdong Freecs,LCK,416,459,0.475429,2,1799.980018,111.289371,2023-02-11 07:03:00
16,DRX,LCK,565,507,0.527052,-1,1699.943287,100.415854,2023-02-10 13:14:00
17,BRION,LCK,171,292,0.369330,-1,1679.939360,98.946035,2023-02-10 10:00:00
22,Nongshim RedForce,LCK,109,165,0.397810,-5,1646.080703,116.011535,2023-02-11 07:03:00


In [6]:
rating.loc[(rating['League'] == 'LPL') & (rating['last_game_date'].dt.year == 2023)]

,Team,League,Win,Loss,WinRate,Streak,r,RD,last_game_date
3,JD Gaming,LPL,385,284,0.575486,7,1990.271298,222.580624,2023-02-09 12:06:00
7,LNG Esports,LPL,391,398,0.495564,1,1875.871382,153.063441,2023-02-08 13:17:00
11,Invictus Gaming,LPL,643,482,0.571556,2,1774.600351,165.961164,2023-02-07 10:16:00
12,EDward Gaming,LPL,709,440,0.617058,2,1766.824038,149.959849,2023-02-09 10:46:00
15,Top Esports,LPL,394,299,0.568543,3,1746.061142,152.440230,2023-02-10 13:08:00
18,Oh My God,LPL,446,532,0.456033,-1,1672.458812,150.370955,2023-02-08 13:17:00
19,Weibo Gaming,LPL,282,241,0.539197,2,1665.920945,154.385754,2023-02-06 10:00:00
30,Bilibili Gaming,LPL,226,229,0.496703,-1,1542.002561,133.373481,2023-02-08 10:51:00
36,Team WE,LPL,564,511,0.524651,1,1505.702401,133.390557,2023-02-08 10:51:00
37,Ninjas in Pyjamas.CN,LPL,144,193,0.427300,-2,1503.160399,141.516514,2023-02-09 10:46:00


In [7]:
rating.loc[(rating['League'] == 'LEC') & (rating['last_game_date'].dt.year == 2023)]

,Team,League,Win,Loss,WinRate,Streak,r,RD,last_game_date
25,MAD Lions,LEC,307,313,0.495161,3,1605.006785,157.023390,2023-02-06 20:07:00
27,Team Vitality,LEC,159,200,0.442897,2,1574.935375,170.199513,2023-02-06 21:13:00
40,G2 Esports,LEC,546,418,0.566390,-1,1490.302471,154.693505,2023-02-06 20:07:00
44,SK Gaming,LEC,174,197,0.469003,2,1477.582241,178.201235,2023-02-06 19:11:00
66,Team BDS,LEC,12,33,0.266667,1,1359.338076,147.424119,2023-02-06 17:08:00
85,KOI (Spanish Team),LEC,4,5,0.444444,1,1289.242185,160.167318,2023-02-06 18:15:00
103,Astralis,LEC,214,219,0.494226,-1,1245.604540,166.797874,2023-02-06 18:15:00
112,Team Heretics,LEC,4,5,0.444444,-2,1217.138360,146.465874,2023-02-06 17:08:00
183,Fnatic,LEC,752,476,0.612378,-4,1121.381971,146.981842,2023-02-06 19:11:00
273,Excel Esports,LEC,62,102,0.378049,-6,970.332902,156.391245,2023-02-06 21:13:00


In [8]:
rating.loc[(rating['League'] == 'LCS') & (rating['last_game_date'].dt.year == 2023)]

,Team,League,Win,Loss,WinRate,Streak,r,RD,last_game_date
10,FlyQuest,LCS,177,216,0.450382,6,1791.288336,190.073646,2023-02-10 22:07:00
26,Evil Geniuses.NA,LCS,175,150,0.538462,1,1604.770621,188.418044,2023-02-10 23:59:00
32,Cloud9,LCS,582,469,0.553758,-1,1534.415830,188.194872,2023-02-10 23:59:00
41,100 Thieves,LCS,194,228,0.459716,-1,1489.142416,188.403086,2023-02-10 22:07:00
97,TSM,LCS,690,451,0.604733,1,1261.559746,176.840535,2023-02-10 23:00:00
139,Golden Guardians,LCS,85,156,0.352697,2,1175.135497,189.828867,2023-02-11 02:03:00
142,Team Liquid,LCS,491,357,0.579009,1,1172.268858,187.062088,2023-02-11 01:03:00
232,Counter Logic Gaming,LCS,324,346,0.483582,-4,1022.109236,193.294073,2023-02-10 23:00:00
259,Immortals,LCS,164,161,0.504615,-1,986.569726,192.133619,2023-02-11 02:03:00
536,Dignitas,LCS,203,274,0.425577,-10,685.939663,223.600224,2023-02-11 01:03:00


In [9]:
rating.loc[(rating['League'].isin(['LCK', 'LPL', 'LEC', 'LCS'])) & (rating['last_game_date'].dt.year == 2023)]

,Team,League,Win,Loss,WinRate,Streak,r,RD,last_game_date
0,Gen.G,LCK,568,386,0.595388,4,2187.255923,121.438602,2023-02-09 12:37:00
1,T1,LCK,859,439,0.661787,3,2122.548742,138.836229,2023-02-08 12:32:00
3,JD Gaming,LPL,385,284,0.575486,7,1990.271298,222.580624,2023-02-09 12:06:00
4,Dplus KIA,LCK,416,174,0.705085,-2,1941.128447,122.457776,2023-02-09 10:03:00
5,Liiv SANDBOX,LCK,201,215,0.483173,1,1926.325018,92.311851,2023-02-10 13:14:00
6,KT Rolster,LCK,592,411,0.590229,2,1878.319323,103.767547,2023-02-09 10:03:00
7,LNG Esports,LPL,391,398,0.495564,1,1875.871382,153.063441,2023-02-08 13:17:00
8,Hanwha Life Esports,LCK,404,411,0.495706,1,1867.243762,109.264336,2023-02-10 10:00:00
9,Kwangdong Freecs,LCK,416,459,0.475429,2,1799.980018,111.289371,2023-02-11 07:03:00
10,FlyQuest,LCS,177,216,0.450382,6,1791.288336,190.073646,2023-02-10 22:07:00


In [10]:
leagues = ['LCK', 'LPL', 'LEC', 'LCS']
for league in leagues:
    print(league)
    lst = rating.loc[(rating['League'] == league) & (rating['last_game_date'].dt.year == 2023), 'Team'].unique()
    print(', '.join(map(lambda x: f"'{x}'", lst)))

LCK
'Gen.G', 'T1', 'Dplus KIA', 'Liiv SANDBOX', 'KT Rolster', 'Hanwha Life Esports', 'Kwangdong Freecs', 'DRX', 'BRION', 'Nongshim RedForce'
LPL
'JD Gaming', 'LNG Esports', 'Invictus Gaming', 'EDward Gaming', 'Top Esports', 'Oh My God', 'Weibo Gaming', 'Bilibili Gaming', 'Team WE', 'Ninjas in Pyjamas.CN', 'ThunderTalk Gaming', 'Royal Never Give Up', 'Rare Atom', 'FunPlus Phoenix', 'Ultra Prime', 'LGD Gaming', 'Anyone's Legend'
LEC
'MAD Lions', 'Team Vitality', 'G2 Esports', 'SK Gaming', 'Team BDS', 'KOI (Spanish Team)', 'Astralis', 'Team Heretics', 'Fnatic', 'Excel Esports'
LCS
'FlyQuest', 'Evil Geniuses.NA', 'Cloud9', '100 Thieves', 'TSM', 'Golden Guardians', 'Team Liquid', 'Counter Logic Gaming', 'Immortals', 'Dignitas'


In [13]:
team_names = {
    'T1': 'T1',
    'GEN': 'Gen.G',
    'DK': 'Dplus KIA',
    'LSB': 'Liiv SANDBOX',
    'KT': 'KT Rolster',
    'HLE': 'Hanwha Life Esports',
    'BRO': 'BRION',
    'DRX': 'DRX',
    'NS': 'Nongshim RedForce',
    'KDF': 'Kwangdong Freecs',
}
lst_team = []
for name in team_names.values():
    team_id = get_team_id(teams_id, name)
    lst_team.append(teams[team_id])

columns = ['Team1', 'Rating1', 'Winprob1', 'Winprob2', 'Rating2', 'Team2']
probs = pd.DataFrame(columns=columns)
for (team1, team2) in permutations(lst_team, 2):
    lst = [
        team1.name, team1.r, team1.get_win_prob(team2) * 100,
        team2.get_win_prob(team1) * 100, team2.r, team2.name
    ]
    df = pd.DataFrame(data=[lst], columns=columns)
    probs = pd.concat([probs, df], ignore_index=True)
probs

,Team1,Rating1,Winprob1,Winprob2,Rating2,Team2
0,T1,2122.548742,41.397324,58.439991,2187.255923,Gen.G
1,T1,2122.548742,72.579851,27.774417,1941.128447,Dplus KIA
2,T1,2122.548742,74.724863,26.237517,1926.325018,Liiv SANDBOX
3,T1,2122.548742,79.172381,21.643860,1878.319323,KT Rolster
4,T1,2122.548742,80.035848,20.670772,1867.243762,Hanwha Life Esports
...,...,...,...,...,...,...
85,Kwangdong Freecs,1743.014088,32.304914,67.397413,1878.319323,KT Rolster
86,Kwangdong Freecs,1743.014088,33.723233,66.077950,1867.243762,Hanwha Life Esports
87,Kwangdong Freecs,1743.014088,58.574645,41.616493,1679.939360,BRION
88,Kwangdong Freecs,1743.014088,55.878423,44.246322,1699.943287,DRX


In [15]:
matches = [
    ('NS', 'KDF'),
    ('GEN', 'KT'),
    ('DK', 'HLE'),
    ('T1', 'DRX'),
]

columns = ['Team1', 'Rating1', 'Winprob1', 'Winprob2', 'Rating2', 'Team2']
probs = pd.DataFrame(columns=columns)
for (team1_name, team2_name) in matches:
    team1 = teams[get_team_id(teams_id, team_names[team1_name])]
    team2 = teams[get_team_id(teams_id, team_names[team2_name])]
    lst = [
        team1.name, team1.r, team1.get_win_prob(team2) * 100,
        team2.get_win_prob(team1) * 100, team2.r, team2.name
    ]
    df = pd.DataFrame(data=[lst], columns=columns)
    probs = pd.concat([probs, df], ignore_index=True)
probs

,Team1,Rating1,Winprob1,Winprob2,Rating2,Team2
0,Nongshim RedForce,1708.629032,45.399234,54.568405,1743.014088,Kwangdong Freecs
1,Gen.G,2187.255923,84.411106,15.984219,1878.319323,KT Rolster
2,Dplus KIA,1941.128447,59.912949,40.217092,1867.243762,Hanwha Life Esports
3,T1,2122.548742,91.034493,9.742358,1699.943287,DRX
